In [19]:
import os

# Add OpenAI library
import openai

# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

In [21]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [22]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [9]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

## Add items to vector store

In [52]:
from langchain_chroma import Chroma

vector_store_ellipsis = Chroma(
    collection_name="Ellipsis-Care-Docs",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Ellipsis-Care/Ellipsis-Care-Chroma-Vector-DB",  # Where to save data locally, remove if not neccesary
)

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

def upsert_pdf_content(pdf_folder_path):
        """
        This method is responsible for upserting PDF content.
        It loads the PDF file, splits the content into chunks, and then upserts the chunks into VecDB.
        """
        documents = []
        for file in os.listdir(pdf_folder_path):
            if file.endswith(".pdf"):
                pdf_path = os.path.join(pdf_folder_path, file)
                loader = PyPDFLoader(pdf_path)
                documents.extend(loader.load())
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
        docs = text_splitter.split_documents(documents)
        return docs

In [54]:
pdf_folder_path =  "/Users/mac/Documents/Ellipsis-Care/data"
splits = upsert_pdf_content(pdf_folder_path)

In [55]:
print(splits[30].page_content)

awareness for the problem of chronic diseases? Is it to build support for the development of an action framework to prevent and control chronic diseases? Is it to promote the already developed action framework to prevent and control chronic diseases? It is important to identify specifically a goal and objectives for the advocating campaign. Establish a clear long-term goal and SMART (specific, measurable, achievable, relevant and timely) objectives at the beginning of your advocacy work.   Step 2: Identify target audiences In advocacy work, the two main audiences will usually be decision-makers and influencers. The more specific you are in identifying the audience, the more effective your communications will be.   A. Potential decision-makers: B. Potential influencers: 1. Government (ministries and parliament): • presidents and prime ministers • health ministers and their deputies • budgetary decision makers (e.g. ministers of finance) • ministers of related sectors and their deputies


In [60]:
splits[0]

Document(metadata={'source': '/Users/mac/Documents/Ellipsis-Care/data/Action Framework for the Prevention and Control of Chronic Disease.pdf', 'page': 0}, page_content='1  Draft version 4, 3 August 2006.          WHO Action Framework  for the  Prevention and Control  of  Chronic Diseases    - Core package -')

In [56]:
# uuids = [str(uuid4()) for _ in range(len(splits))]

In [57]:
len(splits)

2227

In [61]:
import time
from uuid import uuid4

batch_size = 100  # Adjust this batch size based on your rate limit
delay = 70  # Delay in seconds between batches

for i in range(0, len(splits), batch_size):
    batch = splits[i:i+batch_size]
    uuids = [str(uuid4()) for _ in range(len(batch))]
    print(f"Upserting {i} documents")
    vector_store_ellipsis.add_documents(documents=batch, ids=uuids)
    time.sleep(delay)  # Delay to prevent hitting rate limits


Upserting 0 documents


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [51]:
splits[5]

Document(metadata={'source': '/Users/mac/Documents/Ellipsis-Care/data/Action Framework for the Prevention and Control of Chronic Disease.pdf', 'page': 3}, page_content='and registered for free cataract surgery at the Madwai Aravind Eye Hospital the following week. The programme even covered transport costs. “A bus picked me up with seven other cataract patients and drove us to the hospital,” she says. Some 70% of Aravind’s eye patients are charity cases; the 30% who are paying customers support these free sight-restoring operations. The hospital also sells abroad three quarters of the lenses it produces, to help finance its activities. Now 67 years old, Kuzhanthiammal successfully underwent surgery on her other eye a few months ago. “These artificial lenses are a miracle. It’s like waking up with your problems gone,” she joyfully explains.')